In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
import os,sys
import numpy as np
import matplotlib.pyplot as plt
import quantities as pq
import sciunit
import neuronunit
from neuronunit import aibs
from neuronunit.models.reduced import ReducedModel

In [2]:
# This example is from https://github.com/OpenSourceBrain/IzhikevichModel.
IZHIKEVICH_PATH = '/home/mnt/osb/IzhikevichModel' # Replace this the path to your 
                                                                       # working copy of 
                                                                       # github.com/OpenSourceBrain/IzhikevichModel.  
LEMS_MODEL_PATH = os.path.join(IZHIKEVICH_PATH,'NeuroML2/LEMS_2007One.xml')

In [3]:
import quantities as pq
from neuronunit import tests as nu_tests, neuroelectro
neuron = {'nlex_id': 'nifext_50'} # Layer V pyramidal cell
tests = []

dataset_id = 354190013  # Internal ID that AIBS uses for a particular Scnn1a-Tg2-Cre 
                        # Primary visual area, layer 5 neuron.
observation = aibs.get_observation(dataset_id,'rheobase')
tests += [nu_tests.RheobaseTest(observation=observation)]
    
test_class_params = [(nu_tests.InputResistanceTest,None),
                     (nu_tests.TimeConstantTest,None),
                     (nu_tests.CapacitanceTest,None),
                     (nu_tests.RestingPotentialTest,None),
                     (nu_tests.InjectedCurrentAPWidthTest,None),
                     (nu_tests.InjectedCurrentAPAmplitudeTest,None),
                     (nu_tests.InjectedCurrentAPThresholdTest,None)]

for cls,params in test_class_params:
    observation = cls.neuroelectro_summary_observation(neuron)
    tests += [cls(observation,params=params)]
    
def update_amplitude(test,tests,score):
    rheobase = score.prediction['value']
    for i in [5,6,7]:
        print(tests[i])
        # Set current injection to just suprathreshold
        tests[i].params['injected_square_current']['amplitude'] = rheobase*1.01 
    
hooks = {tests[0]:{'f':update_amplitude}}
suite = sciunit.TestSuite("vm_suite",tests,hooks=hooks)

INFO:allensdk.api.api:Downloading URL: http://api.brain-map.org/api/v2/data/query.json?q=model::Specimen,rma::criteria,[id$eq'354190013'],rma::include,structure,donor(transgenic_lines),specimen_tags,cell_soma_locations,ephys_features,data_sets,neuron_reconstructions,cell_reporter,rma::options[num_rows$eq'all']
INFO:allensdk.api.api:Downloading URL: http://api.brain-map.org/api/v2/data/query.json?q=model::EphysSweep,rma::criteria,[specimen_id$eq354190013],rma::options[num_rows$eq'all']


Getting data values from neuroelectro.org
http://www.neuroelectro.org/api/1/nes/?e__name=Input+Resistance&nlex=nifext_50
Getting data values from neuroelectro.org
http://www.neuroelectro.org/api/1/nes/?e__name=Membrane+Time+Constant&nlex=nifext_50
Getting data values from neuroelectro.org
http://www.neuroelectro.org/api/1/nes/?e__name=Cell+Capacitance&nlex=nifext_50
Getting data values from neuroelectro.org
http://www.neuroelectro.org/api/1/nes/?e__name=Resting+membrane+potential&nlex=nifext_50
Getting data values from neuroelectro.org
http://www.neuroelectro.org/api/1/nes/?e__name=Spike+Half-Width&nlex=nifext_50
Getting data values from neuroelectro.org
http://www.neuroelectro.org/api/1/nes/?e__name=Spike+Amplitude&nlex=nifext_50
Getting data values from neuroelectro.org
http://www.neuroelectro.org/api/1/nes/?e__name=Spike+Threshold&nlex=nifext_50


In [4]:
model = ReducedModel(LEMS_MODEL_PATH,name='vanilla')

In [5]:
SUO = '/home/mnt/scidash/sciunitopt'
if SUO not in sys.path:
    sys.path.append(SUO)

In [6]:
from types import MethodType
def optimize(self,model,rov):
    best_params = None
    best_score = None
    from deap_config_simple_sum import deap_capsule
    dc=deap_capsule()
    pop_size=12
    ngen=5                                  
    pop = dc.sciunit_optimize(self,LEMS_MODEL_PATH,pop_size,ngen,rov,
                                                         NDIM=2,OBJ_SIZE=2)
    return pop#(best_params, best_score, model)

my_test = tests[0]
my_test.verbose = False
my_test.optimize = MethodType(optimize, my_test) # Bind to the score.

In [7]:
rov = np.linspace(-100,-40,1000)
pop = my_test.optimize(model,rov)
#print('pareto front top value in pf hall of fame')
#print('best params',best_params,'best_score',best_score, 'model',model)

Start of evolution
V_rest = -91.9; SortKey = 0.505
V_rest = -54.2; SortKey = 0.230
V_rest = -70.3; SortKey = 0.998
V_rest = -60.9; SortKey = 0.600
V_rest = -94.4; SortKey = 0.453
V_rest = -49.9; SortKey = 0.000
V_rest = -54.3; SortKey = 0.230
V_rest = -73.3; SortKey = 0.911
V_rest = -86.3; SortKey = 0.600
V_rest = -45.9; SortKey = 0.000
V_rest = -98.5; SortKey = 0.386
V_rest = -43.7; SortKey = 0.000
Evaluated individuals 12
-- Generation 0 --
V_rest = -60.9; SortKey = 0.600
V_rest = -60.9; SortKey = 0.600
V_rest = -70.6; SortKey = 0.998
V_rest = -73.3; SortKey = 0.911
V_rest = -91.9; SortKey = 0.505
V_rest = -86.8; SortKey = 0.600
V_rest = -69.7; SortKey = 0.966
V_rest = -94.4; SortKey = 0.453
V_rest = -49.9; SortKey = 0.000
V_rest = -86.3; SortKey = 0.600
V_rest = -60.9; SortKey = 0.600
  Evaluated 11 individuals
-- Generation 1 --
V_rest = -73.3; SortKey = 0.911
V_rest = -60.9; SortKey = 0.600
V_rest = -70.6; SortKey = 0.998
V_rest = -69.7; SortKey = 0.966
V_rest = -91.6; SortKey = 0

In [13]:
print("%.2f mV" % np.mean([p[0] for p in pop]))

-70.63 mV
